**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import os

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

/home/michael/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

(you can use Markdown and Latex)

The function act takes the learned action (self.learned_act(s)) which follows the learned policy with probability $P(a_t=\pi(a_t\mid s_t)\mid s_t) = 1-\epsilon$.

The epsilon is essential because it allows the agent to sometimes not respect the policy, hence avoiding bad learned policies.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4 #the 4 amounts for walls width
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16 # sets step size
        # sets the array of possible steps
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        # paints walls in white
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256 # paints position in white
        # paints the rest of the board in black
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        
        #the position matrix is reset after each action
        self.position = np.zeros((self.grid_size, self.grid_size))
        
        #sets walls (forbidden areas)
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3: #walks back if hits a wall
                self.x = self.x-1
            else:
                self.x = self.x + 1 #move RIGHT
        elif action == 1:
            if self.x == 2: #walks back if hits a wall
                self.x = self.x+1
            else:
                self.x = self.x-1 #move LEFT
        elif action == 2:
            if self.y == self.grid_size - 3: #walks back if hits a wall
                self.y = self.y - 1
            else:
                self.y = self.y + 1 #move UP
        elif action == 3:
            if self.y == 2: #walks back if hits a wall
                self.y = self.y + 1
            else:
                self.y = self.y - 1 #move DOWN
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1 #move to next time
        reward = self.board[self.x, self.y] #get the reward (cheese or poison)
        self.board[self.x, self.y] = 0 #cheese is now eaten at the current position
        game_over = self.t > self.max_time 
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board array contains the walls (2 pixels at each border top, right, left, bottom), but also the rewards (or penalties) which are represented by good (positive reward) and poisonous (negative reward) cheese.

The position array contains -1 where the the rat cannot go (the walls), zeros everywhere except where the rat is, where the position is 1.

Both arrays are updated at each action.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass
   
    def learned_act(self,s):
        
        a = np.random.randint(0, self.n_action, size=1)[0]
        
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            #state : 5x5 np.array

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
video = os.path.join(os.path.expanduser('~'),'Documents','MVA','DeepLearning','TP4','random0.mp4')
HTML(data=display_videos(video))

Win/lose count 8.0/11.0. Average score (-3.0)
Win/lose count 9.0/17.0. Average score (-5.5)
Win/lose count 10.0/18.0. Average score (-6.33333333333)
Win/lose count 8.0/14.0. Average score (-6.25)
Win/lose count 12.0/20.0. Average score (-6.6)
Win/lose count 9.0/14.0. Average score (-6.33333333333)
Win/lose count 10.5/11.0. Average score (-5.5)
Win/lose count 9.0/9.0. Average score (-4.8125)
Win/lose count 9.0/17.0. Average score (-5.16666666667)
Win/lose count 7.0/10.0. Average score (-4.95)
Win/lose count 5.5/9.0. Average score (-4.81818181818)
Win/lose count 6.0/15.0. Average score (-5.16666666667)
Win/lose count 9.0/15.0. Average score (-5.23076923077)
Win/lose count 9.5/20.0. Average score (-5.60714285714)
Win/lose count 8.0/14.0. Average score (-5.63333333333)
Win/lose count 7.5/16.0. Average score (-5.8125)
Win/lose count 8.0/9.0. Average score (-5.52941176471)
Win/lose count 8.5/8.0. Average score (-5.19444444444)
Win/lose count 12.0/12.0. Average score (-4.92105263158)
Win/lose

KeyboardInterrupt: 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




By Bellman equation, we have:

\begin{equation}
\begin{aligned}
Q^\pi(s,a) &= \sum\limits_{s'} P^a_{s,s'}[R^a_{s,s'} + \gamma \sum\limits_{a'} \pi(s',a')Q^\pi(s',a')] \\
           &= \sum\limits_{s'} p(s'\mid s,a)[R^a_{s,s'} + \gamma \sum\limits_{a'} p(s',a'\mid s,a)Q^\pi(s',a')]
\end{aligned}
\end{equation}

Taking the conditional expectancy of this equation conditionned on the joint probability $(s',a')\sim p(\cdot \mid s,a)$, we get:

\begin{equation}
\begin{aligned}
    Q^\pi (s,a) &= E_{(s',a')\sim p(.\mid s,a)}(Q^\pi (s,a)) \\
                &= E_{(s',a')\sim p(.\mid s,a)}\left[\sum\limits_{s'}p(s'\mid s,a)R^a_{s,s'}\right] + \gamma E_{(s',a')\sim p(.\mid s,a)}\left[\sum\limits_{s'} p(s'\mid s,a)\sum\limits_{a'} \pi(s',a')Q^\pi(s',a')\right] \\
                &= E_{(s',a')\sim p(.\mid s,a)}\left[\sum\limits_{s'}\sum\limits_{a'}p(s',a'\mid s,a)R^a_{s,s'}\right] + \gamma E_{(s',a')\sim p(.\mid s,a)}\left[\sum\limits_{s'} \sum\limits_{a'}p(s',a'\mid s,a) \pi(s',a')Q^\pi(s',a')\right] \\
                &=  r(s,a) + \gamma E_{(s',a')\sim p(.\mid s,a)}[\sum\limits_{s'}\sum\limits_{a'} p(s',a'\mid s,a) \pi(s',a')Q^\pi(s',a')]\\                                                         
                &= r(s,a) + \sum\limits_{s'}\gamma E_{(s',a')\sim p(.\mid s,a)}[p(s'\mid s,a)\sum\limits_{a'} \pi(s',a')Q^\pi(s',a')]\\  
\end{aligned}
\end{equation}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [80]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        self.memory = self.memory[-self.max_memory:]
        pass

    def random_access(self):
        idx = np.random.randint(0,len(self.memory))        
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [81]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [82]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s,axis=0)))
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        #store transition (no preprocess)
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
                
        input_states = np.zeros((self.batch_size, 5,5,self.n_state)) #5x5 square centered on the agent's position
        # n_state = 2 car poison et fromage
        target_q = np.zeros((self.batch_size, 4)) #4 actions possible? target_q = Q(phi_j,a_j,theta)
        
        for i in range(self.batch_size):
            [s_, n_s_, a_, r_, game_over_] = self.memory.random_access()
            input_states[i] = s_
            if game_over_:
                ######## FILL IN
                target_q[i,:] = r_
                
                pass
            else:
                ######## FILL IN
                n_s_ = np.expand_dims(n_s_, axis=0)
                target_q[i,:] = r_ + self.discount*self.model.predict(n_s_)
                
                pass
        ######## FILL IN        
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q) #perform gradient descent


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, lr=0.1, *args,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Dense(25,input_shape=(5,5,self.n_state,), activation='relu',))
        model.add(Flatten())
#         model.add(Dropout(0.55))
        model.add(Dense(4,activation='relu'))
        
        model.compile(sgd(lr=lr, decay=1e-2, momentum=0.0), "mse")
        self.model = model

In [83]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(.1, size, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

epoch_train = 20
HTML(display_videos('fc_train%d.mp4'%epoch_train))

Epoch 000/050 | Loss 0.0078 | Win/lose count 2.0/6.0 (-4.0)
Epoch 001/050 | Loss 0.0706 | Win/lose count 4.0/9.0 (-5.0)


KeyboardInterrupt: 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [84]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32,(3,3),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Dropout(0.3))
        model.add(Conv2D(16,(2,2),activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4,activation='relu'))
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [56]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(.1,size, epsilon = 0.2, memory_size=1000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
epoch_disp = 30
HTML(display_videos('cnn_train%d.mp4'%epoch_disp))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 3, 3, 32)          608       
_________________________________________________________________
dropout_33 (Dropout)         (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 2, 2, 16)          2064      
_________________________________________________________________
dropout_34 (Dropout)         (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 4)                 260       
Total params: 2,932
Trainable params: 2,932
Non-trainable params: 0
_________________________________________________________________
(5, 5, 2

KeyboardInterrupt: 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [85]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

TypeError: __init__() got multiple values for keyword argument 'lr'

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [86]:
def train_explore(agent,env,epoch,decay_factor=1,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        agent.set_epsilon(decay_factor*agent.epsilon)
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1,train=False):
        grid_size = grid_size+4 #the 4 amounts for walls width
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16 # sets step size
        # sets the array of possible steps
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
        self.train = train

        
    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        reward = 0
        self.get_frame(int(self.t))
        
        #the position matrix is reset after each action
        self.position = np.zeros((self.grid_size, self.grid_size))
        
        #sets walls (forbidden areas)
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3: #walks back if hits a wall
                self.x = self.x-1
            else:
                self.x = self.x + 1 #move RIGHT
        elif action == 1:
            if self.x == 2: #walks back if hits a wall
                self.x = self.x+1
            else:
                self.x = self.x-1 #move LEFT
        elif action == 2:
            if self.y == self.grid_size - 3: #walks back if hits a wall
                self.y = self.y - 1
            else:
                self.y = self.y + 1 #move UP
        elif action == 3:
            if self.y == 2: #walks back if hits a wall
                self.y = self.y + 1
            else:
                self.y = self.y - 1 #move DOWN
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1 #move to next time
        reward = 0
        if self.train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0 #cheese is now eaten at the current position
        game_over = self.t > self.max_time 
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                               self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [58]:
# Training
epochs_train = 20
decay_factor = 0.85
lr = 0.1
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent = DQN_CNN(lr,size, epsilon = 1, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train,decay_factor, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 3, 3, 32)          608       
_________________________________________________________________
dropout_35 (Dropout)         (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 2, 2, 16)          2064      
_________________________________________________________________
dropout_36 (Dropout)         (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 260       
Total params: 2,932
Trainable params: 2,932
Non-trainable params: 0
_________________________________________________________________
(5, 5, 2

KeyboardInterrupt: 

In [88]:
# Training
lr = 0.1
epochs_train = 50
epoch_display = 30

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3,train=True)
agent = DQN_CNN(lr,size, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train,decay_factor= 0.85, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore%d.mp4'%epoch_display))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           (None, 3, 3, 32)          896       
_________________________________________________________________
dropout_59 (Dropout)         (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 2, 2, 16)          2064      
_________________________________________________________________
dropout_60 (Dropout)         (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 4)                 260       
Total params: 3,220
Trainable params: 3,220
Non-trainable params: 0
_________________________________________________________________
Epoch 00

KeyboardInterrupt: 

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***